In [2]:
import os
import cv2
from ultralytics import YOLO

# Define paths
image_folder = "C:\\Users\\ASUS\\Downloads\\flickr8k"  # Folder containing 100 images
output_label_folder = "C:\\Users\\ASUS\\Downloads\\flickr8k_labels"  # Folder for YOLO format labels
os.makedirs(output_label_folder, exist_ok=True)

# Load pre-trained YOLOv8 model
model = YOLO("yolov8n.pt")  # 'n' version is fast; use 'yolov8m.pt' for better accuracy

# Process each image
for img_name in os.listdir(image_folder):
    if img_name.lower().endswith(('.jpg', '.png', '.jpeg')):  # Check for image files
        img_path = os.path.join(image_folder, img_name)

        # Read image
        img = cv2.imread(img_path)

        # Perform object detection
        results = model(img)

        # Open corresponding label file
        label_file = os.path.join(output_label_folder, img_name.replace(".jpg", ".txt"))
        
        with open(label_file, "w") as f:
            for result in results:
                for box in result.boxes:
                    x1, y1, x2, y2 = box.xyxy[0]  # Bounding box in pixel format
                    class_id = int(box.cls[0])  # Object class
                    confidence = box.conf[0]  # Confidence score

                    # Convert to YOLO format (normalized values)
                    img_height, img_width, _ = img.shape
                    x_center = (x1 + x2) / 2 / img_width
                    y_center = (y1 + y2) / 2 / img_height
                    width = (x2 - x1) / img_width
                    height = (y2 - y1) / img_height

                    # Write annotation to file
                    f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

        print(f"Annotated: {img_name}")

print("✅ Auto-annotation completed. Check 'flickr8k_labels' for labels.")


  0%|          | 0.00/6.25M [00:00<?, ?B/s]


0: 480x640 1 person, 1 backpack, 26.1ms
Speed: 4.0ms preprocess, 26.1ms inference, 14.2ms postprocess per image at shape (1, 3, 480, 640)
Annotated: 871290666_4877e128c0.jpg

0: 640x640 1 cow, 1 elephant, 44.1ms
Speed: 4.0ms preprocess, 44.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)
Annotated: 872135364_8c1e47d163.jpg

0: 416x640 14 persons, 36.1ms
Speed: 5.1ms preprocess, 36.1ms inference, 2.0ms postprocess per image at shape (1, 3, 416, 640)
Annotated: 872512911_ca383b40e4.jpg

0: 640x480 2 persons, 37.0ms
Speed: 3.5ms preprocess, 37.0ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 480)
Annotated: 872622575_ba1d3632cc.jpg

0: 640x576 1 person, 84.7ms
Speed: 16.6ms preprocess, 84.7ms inference, 6.0ms postprocess per image at shape (1, 3, 640, 576)
Annotated: 873633312_a756d8b381.jpg

0: 448x640 1 dog, 78.4ms
Speed: 3.2ms preprocess, 78.4ms inference, 5.1ms postprocess per image at shape (1, 3, 448, 640)
Annotated: 874665322_9ad05c4065.jpg

0: 6

In [3]:
import cv2
import os

# Paths
image_folder = "C:\\Users\\ASUS\\Downloads\\flickr8k"   # Folder containing images
label_folder = "C:\\Users\\ASUS\\Downloads\\flickr8k_labels" # Folder containing YOLO txt files
output_folder = "annotated_images"  # Folder to save visualized images
os.makedirs(output_folder, exist_ok=True)

# Read class names (COCO format)
class_names = ["person", "bicycle", "car", "motorcycle", "airplane", "bus", "train", "truck", "boat", "traffic light", 
               "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat", "dog", "horse", "sheep", "cow", 
               "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella", "handbag", "tie", "suitcase", "frisbee", 
               "skis", "snowboard", "sports ball", "kite", "baseball bat", "baseball glove", "skateboard", "surfboard", 
               "tennis racket", "bottle", "wine glass", "cup", "fork", "knife", "spoon", "bowl", "banana", "apple", 
               "sandwich", "orange", "broccoli", "carrot", "hot dog", "pizza", "donut", "cake", "chair", "couch", 
               "potted plant", "bed", "dining table", "toilet", "tv", "laptop", "mouse", "remote", "keyboard", "cell phone", 
               "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors", "teddy bear", 
               "hair drier", "toothbrush"]

# Process each image
for img_name in os.listdir(image_folder):
    if img_name.lower().endswith(('.jpg', '.png', '.jpeg')):
        img_path = os.path.join(image_folder, img_name)
        label_path = os.path.join(label_folder, img_name.replace(".jpg", ".txt"))

        # Load image
        img = cv2.imread(img_path)
        h, w, _ = img.shape

        # Read annotation file
        if os.path.exists(label_path):
            with open(label_path, "r") as f:
                for line in f.readlines():
                    data = line.strip().split()
                    class_id = int(data[0])  # Class index
                    x_center, y_center, width, height = map(float, data[1:])

                    # Convert YOLO format to pixel values
                    x1 = int((x_center - width / 2) * w)
                    y1 = int((y_center - height / 2) * h)
                    x2 = int((x_center + width / 2) * w)
                    y2 = int((y_center + height / 2) * h)

                    # Draw bounding box
                    color = (0, 255, 0)  # Green color for boxes
                    cv2.rectangle(img, (x1, y1), (x2, y2), color, 2)

                    # Put class name
                    label = class_names[class_id] if class_id < len(class_names) else f"Class {class_id}"
                    cv2.putText(img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

        # Save or display image
        output_path = os.path.join(output_folder, img_name)
        cv2.imwrite(output_path, img)
        cv2.imshow("Annotated Image", img)
        cv2.waitKey(500)  # Show for 500ms

cv2.destroyAllWindows()
print("✅ All annotated images saved in 'annotated_images' folder.")


✅ All annotated images saved in 'annotated_images' folder.
